In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab

    # si usan colab, deben cambiar el token de esta url
    ! mkdir -p ../Data
    # los que usan colab deben modificar el token de estas urls:
    ! wget -O ../Data/tags.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_08_Data_Wrangling/Data/tags.csv?token=AA4GFHL5DGGO66Z3XHVBCCS6V4IRM
    ! wget -O ../Data/movies.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_08_Data_Wrangling/Data/movies.csv?token=AA4GFHNVLL665P4RLTZNUVS6V4IUK
    ! wget -O ../Data/ratings.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_08_Data_Wrangling/Data/ratings.csv?token=AA4GFHKROGU63T3BVY7C6W26V4IXE
    
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Data Wrangling

## Introducción

Data wrangling es el proceso de limpieza y unificación de conjuntos de datos desordenados y complejos para facilitar su acceso, exploración, análisis o modelización posterior.

Las tareas que involucra son
* Limpieza de datos
* Eliminación de registros duplicados
* Transformación de datos
* Discretización de variables
* Detección y filtro de outliers
* Construcción de variables dummies


## Dataset

En esta clase usaremos un dataset con info de películas que disponibiliza datos de movielens (https://movielens.org/).

https://grouplens.org/datasets/movielens/

http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

Este conjunto de datos está conformado por varios archivos:
* **movies**: idPelicula, título y género; 

donde cada registro tiene los datos de una película

* **ratings**: idUsuario, idPelicula, rating, fecha; 

donde cada registro tienen la calificación otorgada por un usuario a una película

* **tags**: idUsuario, idPelicula, tag, fecha; 

donde cada registro tienen el tag que asignó un usuario a una película


## Imports

In [ ]:
import pandas as pd
import numpy as np

## Ejercicio 1  - Importar 

Leamos los datos de movies, ratings y tags desde los archivos
* ../Data/movies.csv
* ../Data/ratings.csv
* ../Data/tags.csv

en las variables 
* data_movies
* data_ratings
* data_tags

Veamos cuántos registros hay en cada DataFrame y de qué tipos son los datos de cada columna. 

Veamos los primeros registros de cada DataFrame para verificar que los datos fueron importados correctamente.

In [ ]:
data_ratings = pd.read_csv("../Data/ratings.csv", sep=",")
print(data_ratings.dtypes)
data_ratings.head(3)

In [ ]:
data_tags = pd.read_csv("../Data/tags.csv", sep=",")
print(data_tags.dtypes)
data_tags.head(3)

In [ ]:
data_movies = pd.read_csv("../Data/movies.csv", sep=",")
print(data_movies.dtypes)
data_movies.head(3)

## Ejercicio 2  - Registros duplicados

**2.a** Veamos si existen registros duplicados en el DataFrame data_tags considerando sólo las columnas "movieId", "tag", marcando como no duplicado la primera ocurrencia de un valor.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html


**2.b** Usemos el método `drop_duplicates` para obtener otro  `DataFrame` sin los casos duplicados considerando sólo las columnas "movieId", "tag". Usemos el método `duplicated` para verificar que el nuevo `DataFrame` efectivamente no tiene valores duplicados.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html

In [ ]:
duplicated_mask = data_tags.duplicated(subset = [ "movieId", "tag"], keep = "first")

print(type(duplicated_mask))

print(any(duplicated_mask))

duplicated_records = data_tags.loc[duplicated_mask]
duplicated_records.sort_values(by=["movieId", "tag"]).head(10)


In [ ]:
data_tags_nodup = data_tags.drop_duplicates(subset = ["movieId", "tag"], keep = "first")
print(data_tags.shape)
print(data_tags_nodup.shape)

dup_mask = data_tags_nodup.duplicated(subset = ["movieId", "tag"], keep = "first")
any(dup_mask)

## Ejercicio 3 - Transformar datos

Construyamos un diccionario que asocie un puntaje a una etiqueta.

Las etiquetas son:

* mala, para puntajes menores a 3;

* regular, para mayor igual a 3 y  menor que 4;

* buena para puntaje mayor o igual que 4

Usemos el método `map` para crear una nueva columna en data (`rating_label`) que tenga las etiquetas asociadas al valor del campo `rating` para cada registro


In [ ]:
np.sort(data_ratings.rating.unique())

In [ ]:
etiquetas = { 0.5: "mala",
             1:  "mala",
             1.5:  "mala",
             2:  "mala",
             2.5:  "mala",
             3: "regular",
             3.5: "regular",
             4: "buena",
             4.5: "buena",
             5: "buena"    
}

In [ ]:
data_ratings["rating_label"] = data_ratings.rating.map(etiquetas)
data_ratings.head(3)

Nota: esto ya sabíamos resolverlo usando máscaras booleanas

## Ejercicio 4 - Reemplazar valores

El método `replace` ofrece varias formas de efectuar reemplazos sobre una serie de Pandas:

* Un valor viejo por un valor nuevo.
    
* Una lista de valores viejos por un valor nuevo.
    
* Una lista de valores viejos por una lista de valores nuevos.
    
* Un diccionario que mapee valores nuevos y viejos.


https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html

**4.a - Una lista de valores viejos por un valor nuevo** 

Veamos cuáles son los tags que están asignados a una única película. 

Reemplacemos ese valor por "tag_que_no_funciona" y eliminemos registros duplicados considerando los campos "userId",  "movieId", "tag".

Ayuda: `value_counts`

In [ ]:
tag_counts = data_tags.tag.value_counts()
tag_counts_1_mask = tag_counts == 1
tags_counts_1 = tag_counts.loc[tag_counts_1_mask]
print(len(tag_counts))
print(len(tags_counts_1))
tags_counts_1

In [ ]:
data_tags_frecuentes =  data_tags.replace(tags_counts_1.index, "tag_que_no_funciona")
data_tags_frecuentes

In [ ]:
data_tags_frecuentes_nodup = data_tags_frecuentes.drop_duplicates(subset = ["userId", "movieId", "tag"], keep = "first")
print(data_tags_frecuentes_nodup.shape)
print(data_tags_frecuentes.shape)

**4.b - Una lista de valores viejos por una lista de valores nuevos**

Reemplacemos cada valor de tag, por la primera palabra que lo compone.

Para eso, creamos una serie con valores únicos con el valor del campo tag. 

Contruimos otra instancia de Series donde cada elemento sea la primera palabra del objeto Series anterior. Ayuda: listas por comprensión y `split`

Usamos replace para campiar el valor de cada tag por su primera palabra.


In [ ]:
tags_unique = data_tags.tag.unique()
#print(tags_unique)
print(len(tags_unique))
tags_primera_palabra = [x.split()[0] for x in tags_unique]
#print(tags_primera_palabra)
print(len(tags_primera_palabra))

In [ ]:
data_tags_resumen = data_tags.replace(tags_unique, tags_primera_palabra)
print(data_tags.head(10))
print(data_tags_resumen.head(10))

**4.c - Un diccionario que mapee valores nuevos y viejos**

Reemplacemos los valores de tags 
* "Al Pacino" por "Pacino"
* "Leonardo DiCaprio" por "DiCaprio"
* "Tom Hanks" por "Hanks"
* "Martin Scorsese" por "Scorsese"

Contemos cuantas veces aparecen cada uno de los valores a reemplazar, y cuántas los valores de reemplazo. Ayuda: `value_counts`

Construyamos un diccionario con este mapeo y usemos el método `replace`

Volvamos a contar cuántas veces aparecen cada uno de los valores a reemplazar, y cuántas los valores de reemplazo.


In [ ]:
tag_counts = data_tags.tag.value_counts()
print(tag_counts["Al Pacino"])
print(tag_counts["Leonardo DiCaprio"])
print(tag_counts["Tom Hanks"])
print(tag_counts["Martin Scorsese"])

In [ ]:
print("Pacino" in tag_counts.keys())
print("DiCaprio" in tag_counts.keys())
print("Hanks" in tag_counts.keys())
print("Scorsese" in tag_counts.keys())


In [ ]:
replacement = {
    "Al Pacino": "Pacino", 
    "Leonardo DiCaprio": "DiCaprio",
    "Tom Hanks": "Hanks",
    "Martin Scorsese": "Scorsese"
}

data_tags_dict_replacement = data_tags.replace(replacement)

In [ ]:
tag_counts_replacement = data_tags_dict_replacement.tag.value_counts()
print(tag_counts_replacement["Pacino"])
print(tag_counts_replacement["DiCaprio"])
print(tag_counts_replacement["Hanks"])
print(tag_counts_replacement["Scorsese"])

In [ ]:
print("Al Pacino" in tag_counts_replacement.keys())
print("Leonardo DiCaprio" in tag_counts_replacement.keys())
print("Tom Hanks" in tag_counts_replacement.keys())
print("Martin Scorsese" in tag_counts_replacement.keys())


## Ejercicio 5 - Discretizar variables

Vamos a volver a resolver el Ejercicio 3 usando el método `cut`

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html

Defino los valores de corte:

* mala, para puntajes menores a 3;

* regular, para mayor igual a 3 y  menor que 4;

* buena para puntaje mayor o igual que 4


In [ ]:
# valores de corte:
bins_rating = [0, 3, 4, 5.1]
labels_rating = ["mala", "regular", "buena"]

# Obtengo una lista de intervalos
# right=False indice que el extremo derecho del intervalo no está incluído en la categoría
categories = pd.cut(data_ratings.rating, bins_rating, labels = labels_rating, right=False)

data_ratings["rating_label_cut"] = categories
data_ratings.head(10)

## Ejercicio 6 - Detectar y filtrar outliers

No existe un criterio que sea válido en todos los casos para identificar los outliers. El criterio de mayor que el tercer cuartil más 1.5 veces el rango intercuartil o menor que el primer cuartil menos 1.5 veces el rango intercuartil (Q3 - Q1) surge de la distribución normal. En esa distribución el 99.7% de la población se encuentra en el rango definido por la media (poblacional) más menos 3 veces el desvío estándar (poblacional)

**Queremos ver cuáles son las películas que son outliers en cantidad de calificaciones.**

**6.a** Usando data_ratings eliminamos duplicados considerando las columnas "userId", "movieId". Esto lo hacemos para contar sólo una vez los votos de un usuario a una película.

**6.b** Sobre el DataFrame obtenido en el paso anterior, hacemos count agrupado por película. Esto nos da como resultado una instancia de Series que asignamos a la variable movie_votes_count.

**6.c** Calculemos los cuartilos de los valores de movie_votes_count y los valores que usaremos de umbral para determinar outliers.
(Ayuda: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.quantile.html)

**6.d** Filtremos los datos de movie_votes_count excluyendo los outliers. 

**6.e** Comparemos movie_votes_count antes y después del filtro con:
* el método `describe`
* boxplots de seaborn

**6.f** Adicional: Miremos cuáles son los títulos de las cinco películas más votadas que son outliers de cantidad de calificaciones

In [ ]:
#6.a

data_ratings_nodup = data_ratings.drop_duplicates(subset = ["userId", "movieId"], keep = "first")

#6.b
movie_votes_count = data_ratings_nodup.groupby("movieId")["userId"].count()

#6.c
q1 = movie_votes_count.quantile(0.25)
print(q1)
q2 = movie_votes_count.quantile(0.5)
print(q2)
q3 = movie_votes_count.quantile(0.75)
print(q3)


iqr = (q3 - q1) * 1.5

up_threshold = q3 + iqr
low_threshold = q1 - iqr

print(up_threshold)
print(low_threshold)

#6.d
outlier_mask_up = movie_votes_count > up_threshold
outlier_mask_down = movie_votes_count < low_threshold
outlier_mask = np.logical_or(outlier_mask_up, outlier_mask_down)
not_outliers = np.logical_not(outlier_mask)

outliers = movie_votes_count[outlier_mask]

In [ ]:
#6.e
movie_votes_count_filtered = movie_votes_count[not_outliers]
movie_votes_count_filtered.describe()

In [ ]:
movie_votes_count.describe()

In [ ]:
import seaborn as sns
sns.boxplot(x=movie_votes_count.values);

In [ ]:
sns.boxplot(x=movie_votes_count_filtered.values);

In [ ]:
#6.f

outliers_sort = np.sort(outliers.values)[::-1]
print("cantidad de votos ordenados de mayor a menor: ", outliers_sort)

top_5 = outliers_sort[0:5]
print("cantidad de votos máximos: ", top_5)

top_5_outliers_mask = outliers.apply(lambda x: x in top_5)
top_5_outliers_movieId = outliers.loc[top_5_outliers_mask].index
print("ids de las peliculas en el top 5 de votos", top_5_outliers_movieId)

top_5_movieId_mask = data_movies.movieId.apply(lambda x: x in top_5_outliers_movieId)
data_movies.loc[top_5_movieId_mask, "title"]


## Ejercicio 7 - Variables categóricas y dummies

**7.a** Usando el método `get_dummies` con `drop_first = True` agreguemos al DataFrame data_ratings variables dummies que representen las categorias de rating_label

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html


**7.b** Comparemos las variables dummies generadas en el punto anterior con las que se generan usando `drop_first = False`. ¿Cuál es la diferencia? ¿Representan el mismo conjunto de valores posibles?

**7.c** Adicional: Cambienos las categorias que se muestran como resultado de `get_dummies` con `drop_first = True`. Ayuda: https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html#categoricaldtype



In [ ]:
# 7.a

dummies_rating_label = pd.get_dummies(data_ratings.rating_label, prefix='rating_label', drop_first=True)

# agregamos las columnas dummies
data_ratings_with_dummy = data_ratings.join(dummies_rating_label)

# quitamos la columna rating_label que ahora queda representada por las dummies
result = data_ratings_with_dummy.drop(labels = ["rating_label"], axis="columns")
result.head(5)

In [ ]:
# 7.b

dummies_rating_label = pd.get_dummies(data_ratings.rating_label, prefix='rating_label', drop_first=False)
dummies_rating_label

En esta segunda solución (7.b), tenemos una columna para cada categoría de los valores originales. 

Los valores representados son exactamente los mismos que había en la columna original (como en las solución 7.a), pero una de las columnas es redundante porque se puede determinar su valor partiendo de los valores de las otras dos.

Las dos soluciones representan todas la categorias posibles de la variable original. 

Observemos que los valores (0,0,0), (0,1,1), (1,1,0), (1,0,1), (1,1) no representan una categoría en la variable original.

In [ ]:
# 7.c

from pandas.api.types import CategoricalDtype
    
cat_type = CategoricalDtype(categories=["mala", "regular", "buena"], ordered=True)

labels_cat = data_ratings.rating_label.astype(cat_type)

pd.get_dummies(labels_cat, prefix='rating_label', drop_first=True)